# 03_Modeling

## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.metrics import average_precision_score, adjusted_rand_score, roc_auc_score, average_precision_score, silhouette_score
import hdbscan
import umap
import joblib  
import plotly.express as px



## Load data

In [118]:
df = pd.read_csv('../data/processed/windowed_dataset_cleaned.csv')
X = df[df.columns.difference(['is_attack'])]  # Features
y = df['is_attack']  # Labels

## HDBSCAN

In [ ]:
import hdbscan
import pandas as pd
import numpy as np

# -----------------------------
# HDBSCAN clustering
# -----------------------------
hdb = hdbscan.HDBSCAN(
    min_cluster_size=100,#Se recomienda igualar min_samples a min_cluster_size para outlier detection
    min_samples=100, # bajarlo significa tomar mas riesgo y clusterizar mas puntos, subirlo significa ser mas conservador y detectar mas outliers
    cluster_selection_method='eom', # 'leaf' or 'eom' EXCESS of Mass para clusters más grandes y estables
    metric='euclidean'
)
hdb_labels = hdb.fit_predict(X)
print("HDBSCAN clusters:", np.unique(hdb_labels))

HDBSCAN clusters: [-1  0  1  2]


In [157]:
import plotly.express as px
import pandas as pd
import numpy as np

import umap

# -----------------------------
# UMAP embedding
# -----------------------------
reducer = umap.UMAP(
    n_neighbors=30,
    min_dist=0.1,
    n_components=2,
    metric='euclidean',
    random_state=42
)

X_umap = reducer.fit_transform(X)

df_vis = df.copy()
df_vis['UMAP1'] = X_umap[:, 0]
df_vis['UMAP2'] = X_umap[:, 1]
df_vis['cluster_label'] = hdb_labels

df_vis['Cluster'] = df_vis['cluster_label'].map(
    lambda lbl: 'Outlier' if lbl == -1 else f'Cluster {lbl}'
)

cluster_order = ['Outlier'] + [f'Cluster {i}' for i in sorted(df_vis[df_vis['cluster_label'] != -1]['cluster_label'].unique())]
df_vis['Cluster'] = pd.Categorical(df_vis['Cluster'], categories=cluster_order, ordered=True)

hover_cols = ['is_attack', 'n_connections'] 

fig = px.scatter(
    df_vis,
    x='UMAP1',
    y='UMAP2',
    color='Cluster',
    hover_data=hover_cols,  # Ahora sí: columnas del DataFrame
    title="UMAP + HDBSCAN clusters",
    width=900,
    height=700
)

fig.update_traces(marker=dict(size=6, opacity=0.8))
fig.update_layout(legend_title_text='HDBSCAN Clusters')
fig.write_image(
    "../reports/figures/dataset_proportion2.png",
    width=1000,      # ancho en píxeles
    height=600,      # alto en píxeles
    scale=2          # factor de escalado (2 = ~200 DPI)
)
fig.show()

/Users/taxi/miniconda3/envs/consoning/lib/python3.11/site-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.

/Users/taxi/miniconda3/envs/consoning/lib/python3.11/site-packages/sklearn/manifold/_spectral_embedding.py:324: UserWarning:

Graph is not fully connected, spectral embedding may not work as expected.



In [158]:
pd.crosstab(df['is_attack'], hdb_labels)

col_0,-1,0,1,2
is_attack,,,,
0.0,1091,45,22079,444
1.0,222,124,0,0


## Save model

In [176]:
joblib.dump(model, "../models/hdbscan_model.joblib")

NameError: name 'joblib' is not defined